## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## IMDB 데이터셋 준비

In [3]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

# X : 영화평, y : 긍정적/부정적
# 각 문장에서 많이 사용되는 1만개의 단어만 추출. 다시 각 문장에서 32개 단어만 사용.
# 32

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=32)

print(x_train.shape, y_train.shape)

(25000, 32) (25000,)


In [4]:
print(x_train[0])

[  30 5535   18   51   36   28  224   92   25  104    4  226   65   16
   38 1334   88   12   16  283    5   16 4472  113  103   32   15   16
 5345   19  178   32]


In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 정의

In [6]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16) # 각 단어를 16 길이로 임베딩
#         self.rnn = tf.keras.layers.SimpleRNN(32)
#         self.rnn = tf.keras.layers.LSTM(32)
        self.rnn = tf.keras.layers.GRU(32)
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')
    
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 모델 생성

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 학습 루프 동작

In [8]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 9s 11ms/step - loss: 0.5080 - accuracy: 0.7366 - val_loss: 0.4401 - val_accuracy: 0.7920
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3745 - accuracy: 0.8334 - val_loss: 0.4612 - val_accuracy: 0.7832
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3236 - accuracy: 0.8634 - val_loss: 0.4918 - val_accuracy: 0.7776
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2807 - accuracy: 0.8858 - val_loss: 0.5032 - val_accuracy: 0.7774
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2377 - accuracy: 0.9084 - val_loss: 0.5997 - val_accuracy: 0.7713
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2004 - accuracy: 0.9257 - val_loss: 0.6905 - val_accuracy: 0.7618
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.1669 - accuracy: 0.9397 - v